In [1]:
import wctablereader as tablereader
from glob import glob
import pandas as pd
from pathlib import Path
import importlib
from IPython.display import Markdown


In [2]:
importlib.reload(tablereader)
WCOutputs = Path('..')/'..'/'WCOutputs'
wildtype = WCOutputs/'manual'/'wildtype_000000'
wildtype_simout = wildtype / '000000'/'generation_000000'/'000000'/'simOut/'


def simplify(dfs):
    def _maybe_extend(df, meta):
        if df.shape[0] == meta.shape[0]: 
            order = meta.columns.to_list() + df.columns.to_list()
            df = df.join(meta)[order]
        return df

    wide = {k:df for k, df in dfs.items()
              if df.shape[1] > 1}

    narrow = {k:df[df.columns[0]].rename(df.index.name) for k, df in dfs.items()
              if df.shape[1] == 1}
    shapes = [df.shape for df in narrow.values()]
    all_same = all([s==shapes[0] for s in shapes])
    if all_same:
        meta = pd.DataFrame([*narrow.values()]).T
    
    wide = {k: _maybe_extend(df, meta) for k, df in wide.items()}
    return wide


def table_to_frames(filename, do_simplify=True):
    dfs = {}
    tb = tablereader.TableReader( filename )
    for col in tb.columnNames():
        dfs[col] = tb.as_pandas( col )
        dfs[col].index.name = col
        
    if do_simplify: 
        dfs = simplify(dfs)
        
    return dfs

def print_frames(dfs):
    for name, df in dfs.items():
        if df.shape[1] == 0:
            display(Markdown(f'## {name}: empty table'))
        else:
            display(Markdown(f'## {name}:'))
            display(dfs[name])
    return dfs

In [3]:
#Sweep some output, see if/where there are problems
import warnings

tables = glob(str(wildtype_simout/'*'))
for table in tables:
    first_error = True
    try: 
        tb = tablereader.TableReader(table)
        print(f"Success opening {table}")
    except: 
        print(f"!!!!Error opening {table}")
        continue
    for column in tb.columnNames():
        try: 
            warnings.filterwarnings("error")
            df = tb.as_pandas(column)
            warnings.filterwarnings("ignore")
        except Exception as e: 
            if first_error:
                print(f"Error reading column in {table}")
                first_error = False
            print(f"\tError on '{column}'")
            print("\t", e)

Success opening ../../WCOutputs/manual/wildtype_000000/000000/generation_000000/000000/simOut/Mass
Success opening ../../WCOutputs/manual/wildtype_000000/000000/generation_000000/000000/simOut/FBAResults
Success opening ../../WCOutputs/manual/wildtype_000000/000000/generation_000000/000000/simOut/ReplicationData
Success opening ../../WCOutputs/manual/wildtype_000000/000000/generation_000000/000000/simOut/BulkMolecules
Success opening ../../WCOutputs/manual/wildtype_000000/000000/generation_000000/000000/simOut/RnaDegradationListener
Error reading column in ../../WCOutputs/manual/wildtype_000000/000000/generation_000000/000000/simOut/RnaDegradationListener
	Error on 'countRnaDegraded'
	 No attributes match shape for countRnaDegraded.  Returning data without labels.
Success opening ../../WCOutputs/manual/wildtype_000000/000000/generation_000000/000000/simOut/RibosomeData
!!!!Error opening ../../WCOutputs/manual/wildtype_000000/000000/generation_000000/000000/simOut/Daughter2
Success open

In [4]:
tb = tablereader.TableReader("../../WCOutputs/manual/wildtype_000000/000000/generation_000000/000000/simOut/FBAResults")
tb.columnNames()

['simulationStep',
 'targetConcentrations',
 'succinate_flux_sensitivity',
 'externalExchangeFluxes',
 'time',
 'objectiveValue',
 'reducedCosts',
 'reactionFluxes',
 'isocitrate_flux_sensitivity',
 'homeostaticObjectiveValues',
 'kineticObjectiveValues',
 'deltaMetabolites',
 'shadowPrices']

# FBA Results


In [5]:
fba_results = print_frames(table_to_frames(wildtype_simout / 'FBAResults'))

## targetConcentrations:

,simulationStep,time,objectiveValue,2-3-DIHYDROXYBENZOATE[c],2-KETOGLUTARATE[c],4-hydroxybenzoate[c],ACETOACETYL-COA[c],ACETYL-COA[c],ACETYL-P[c],ADENINE[c],...,UDP[c],UMP[c],UNDECAPRENYL-DIPHOSPHATE[c],URIDINE[c],UTP[c],VAL[c],WATER[c],ZN+2[c],ZN+2[p],glycogen-monomer[c]
targetConcentrations,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.000000,0.000000,0.00,0.00,0.000,0.000,0.00,0.0,0.0000,...,0.0,0.000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000
1,1.0,0.200000,0.022852,0.14,0.44,0.052,0.022,0.61,1.1,0.0015,...,1.8,0.024,0.025115,2.1,8.3,4.0,42742.159312,0.168848,0.1,66.026786
2,2.0,0.400000,0.018119,0.14,0.44,0.052,0.022,0.61,1.1,0.0015,...,1.8,0.024,0.025115,2.1,8.3,4.0,42742.159312,0.168848,0.1,66.026786
3,3.0,0.600000,0.017088,0.14,0.44,0.052,0.022,0.61,1.1,0.0015,...,1.8,0.024,0.025115,2.1,8.3,4.0,42742.159312,0.168848,0.1,66.026786
4,4.0,0.800000,0.016130,0.14,0.44,0.052,0.022,0.61,1.1,0.0015,...,1.8,0.024,0.025115,2.1,8.3,4.0,42742.159312,0.168848,0.1,66.026786
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
672,672.0,596.610156,0.300241,0.14,0.44,0.052,0.022,0.61,1.1,0.0015,...,1.8,0.024,0.025115,2.1,8.3,4.0,42742.159312,0.168848,0.1,66.026786
673,673.0,597.503125,0.300712,0.14,0.44,0.052,0.022,0.61,1.1,0.0015,...,1.8,0.024,0.025115,2.1,8.3,4.0,42742.159312,0.168848,0.1,66.026786
674,674.0,598.396094,0.301139,0.14,0.44,0.052,0.022,0.61,1.1,0.0015,...,1.8,0.024,0.025115,2.1,8.3,4.0,42742.159312,0.168848,0.1,66.026786


## externalExchangeFluxes:

,simulationStep,time,objectiveValue,ACET[p],AMMONIUM[c],BETAINE[p],BUTANAL[c],CA+2[p],CARBON-DIOXIDE[p],CARBON-MONOXIDE[p],...,S-ADENOSYL-4-METHYLTHIO-2-OXOBUTANOATE[c],SUC[p],SULFATE[p],THYMINE[p],UNDECAPRENYL-DIPHOSPHATE[p],URACIL[p],UREA[p],WATER[p],XANTHINE[p],ZN+2[p]
externalExchangeFluxes,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
1,1.0,0.200000,0.022852,-0.0,-12.906195,0.0,0.0,-0.219226,22.165272,0.007560,...,0.001700,-0.0,-2.051496,0.0,0.000000,-0.0,0.0,287.528753,-0.0,-0.021913
2,2.0,0.400000,0.018119,-0.0,-14.038226,-0.0,-0.0,0.000042,14.982359,0.000068,...,0.000000,-0.0,-0.119526,-0.0,0.000267,-0.0,-0.0,197.452434,-0.0,-0.000030
3,3.0,0.600000,0.017088,-0.0,-6.054397,-0.0,-0.0,-0.000032,29.206674,-0.000000,...,0.000000,-0.0,-0.116480,-0.0,0.000468,-0.0,-0.0,161.462264,-0.0,0.000044
4,4.0,0.800000,0.016130,-0.0,-6.884775,-0.0,-0.0,0.000041,37.138720,-0.000000,...,0.000000,-0.0,-0.110465,-0.0,0.000468,-0.0,-0.0,99.973080,-0.0,0.000044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
672,672.0,596.610156,0.300241,-0.0,-7.697869,-0.0,-0.0,-0.007545,29.533285,0.000270,...,0.000059,-0.0,-0.180657,-0.0,0.010592,-0.0,-0.0,-71.948301,-0.0,-0.000778
673,673.0,597.503125,0.300712,-0.0,-8.033295,-0.0,-0.0,-0.007485,29.544827,0.000259,...,0.000059,-0.0,-0.181696,-0.0,0.010518,-0.0,-0.0,-70.242888,-0.0,-0.000749
674,674.0,598.396094,0.301139,-0.0,-8.110883,-0.0,-0.0,-0.007523,29.625022,0.000250,...,0.000060,-0.0,-0.180159,-0.0,0.010560,-0.0,-0.0,-70.492354,-0.0,-0.000766


## reducedCosts:

...

,simulationStep,time,objectiveValue,1.1.1.215-RXN,1.1.1.262-RXN,1.1.1.39-RXN,1.1.1.83-RXN,1.8.4.13-RXN-MET/Ox-Thioredoxin/WATER//CPD-8989/Red-Thioredoxin.51.__EG11433-MONOMER,1.8.4.14-RXN-MET/Ox-Thioredoxin/WATER//CPD-8990/Red-Thioredoxin.51.__EG12394-MONOMER,1.8.4.14-RXN-MET/Ox-Thioredoxin/WATER//CPD-8990/Red-Thioredoxin.51.__G7005-MONOMER,...,UDPNACETYLGLUCOSAMACYLTRANS-RXN-R-3-hydroxymyristoyl-ACPs/UDP-N-ACETYL-D-GLUCOSAMINE//UDP-OHMYR-ACETYLGLUCOSAMINE/ACP.86.,UDPNACETYLGLUCOSAMENOLPYRTRANS-RXN,UDPNACETYLMURAMATEDEHYDROG-RXN,UGD-RXN,UNDECAPRENYL-DIPHOSPHATASE-RXN__PGPPHOSPHAB-MONOMER,URA-PHOSPH-RXN,UROGENIIISYN-RXN,URPHOS-RXN (reverse),XANPRIBOSYLTRAN-RXN,XYLULOKIN-RXN
kineticObjectiveValues,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0
1,1.0,0.200000,0.022852,0.0,17.088456,1.0,1.0,1.0,1.0,1.0,...,1.000000,1.000000,1.000000,19.037087,1.000000,0.0,0.999990,0.0,1.000000,1.0
2,2.0,0.400000,0.018119,0.0,0.947322,1.0,1.0,1.0,1.0,1.0,...,0.873780,0.981278,0.992016,0.993350,0.631569,0.0,0.999990,0.0,0.989216,1.0
3,3.0,0.600000,0.017088,0.0,1.000000,0.0,1.0,1.0,1.0,1.0,...,46.950809,0.967201,0.986013,1.000000,0.354535,0.0,0.999990,0.0,0.985932,1.0
4,4.0,0.800000,0.016130,0.0,1.000000,0.0,1.0,1.0,1.0,1.0,...,1.000000,0.967201,0.986013,1.000000,0.354535,0.0,0.999990,0.0,0.970741,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
672,672.0,596.610156,0.300241,0.0,0.340992,0.0,1.0,1.0,1.0,1.0,...,0.946236,0.134877,0.662714,0.192929,0.000000,0.0,0.998618,0.0,0.000000,1.0
673,673.0,597.503125,0.300712,0.0,0.335048,0.0,1.0,1.0,1.0,1.0,...,0.935994,0.140694,0.664983,0.191203,0.000000,0.0,0.998614,0.0,0.000000,1.0
674,674.0,598.396094,0.301139,0.0,0.361787,0.0,1.0,1.0,1.0,1.0,...,0.946745,0.137094,0.663579,0.184986,0.000000,0.0,0.998603,0.0,0.000000,1.0


## deltaMetabolites:

,simulationStep,time,objectiveValue,2-3-DIHYDROXYBENZOATE[c],2-KETOGLUTARATE[c],4-hydroxybenzoate[c],ACETOACETYL-COA[c],ACETYL-COA[c],ACETYL-P[c],ADENINE[c],...,UDP[c],UMP[c],UNDECAPRENYL-DIPHOSPHATE[c],URIDINE[c],UTP[c],VAL[c],WATER[c],ZN+2[c],ZN+2[p],glycogen-monomer[c]
deltaMetabolites,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.200000,0.022852,154.0,482.0,58.0,453.0,668.0,1205.0,2.0,...,1972.0,27.0,28.0,2301.0,10810.0,8221.0,-3037975.0,185.0,110.0,31951.0
2,2.0,0.400000,0.018119,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,-1.0,-636.0,0.0,-1.0,1701.0,3786.0,-2048778.0,1.0,0.0,40352.0
3,3.0,0.600000,0.017088,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-542.0,0.0,0.0,1722.0,3770.0,-1541165.0,0.0,0.0,0.0
4,4.0,0.800000,0.016130,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,-308.0,0.0,0.0,1703.0,3562.0,-659573.0,0.0,-1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
672,672.0,596.610156,0.300241,28.0,88.0,11.0,4.0,121.0,218.0,0.0,...,357.0,-4138.0,5.0,416.0,11161.0,20123.0,8235512.0,34.0,21.0,13082.0
673,673.0,597.503125,0.300712,28.0,87.0,10.0,5.0,120.0,216.0,1.0,...,354.0,-3191.0,5.0,414.0,11171.0,20102.0,8185261.0,33.0,19.0,13002.0
674,674.0,598.396094,0.301139,28.0,87.0,10.0,4.0,121.0,218.0,0.0,...,356.0,-2710.0,5.0,415.0,11418.0,19535.0,8231269.0,34.0,20.0,13073.0


## shadowPrices:

,simulationStep,time,objectiveValue,2-3-DIHYDROXYBENZOATE[c],2-KETOGLUTARATE[c],4-hydroxybenzoate[c],ACETOACETYL-COA[c],ACETYL-COA[c],ACETYL-P[c],ADENINE[c],...,UDP[c],UMP[c],UNDECAPRENYL-DIPHOSPHATE[c],URIDINE[c],UTP[c],VAL[c],WATER[c],ZN+2[c],ZN+2[p],glycogen-monomer[c]
shadowPrices,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
1,1.0,0.200000,0.022852,0.016521,0.009348,0.066231,0.167616,0.163322,4.408507e-03,0.090266,...,0.015637,0.014911,0.341429,0.014142,0.016364,0.013276,2.339608e-05,8.557589e-05,8.557589e-05,0.015145
2,2.0,0.400000,0.018119,0.012988,0.008815,-0.034717,0.132158,0.128033,4.177744e-03,0.093493,...,0.012339,0.011953,0.102618,0.011497,0.012726,0.010965,2.339608e-05,8.557589e-05,8.557589e-05,0.011779
3,3.0,0.600000,0.017088,0.002998,0.001984,-0.045068,-0.712594,-0.713439,8.946009e-04,0.101186,...,0.004325,0.004120,0.119981,0.003890,0.004531,0.002913,2.339608e-05,6.006405e-05,8.557589e-05,-0.015145
4,4.0,0.800000,0.016130,0.002976,0.001946,-0.045094,-0.712592,-0.713438,1.064872e-03,0.101207,...,0.004789,0.004585,0.119642,0.004358,0.004993,0.002899,2.339608e-05,6.055000e-05,8.557589e-05,-0.015145
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
672,672.0,596.610156,0.300241,-0.000078,0.000005,-0.013035,0.021572,0.021572,-8.427633e-08,0.023059,...,0.000066,0.000066,0.038059,0.000066,0.000067,0.000074,2.724538e-08,9.889398e-08,9.889398e-08,-0.000025
673,673.0,597.503125,0.300712,-0.000078,0.000005,-0.013038,0.021578,0.021578,-8.427633e-08,0.023064,...,0.000066,0.000066,0.038067,0.000066,0.000067,0.000074,2.724538e-08,9.889398e-08,9.889398e-08,-0.000025
674,674.0,598.396094,0.301139,-0.000078,0.000005,-0.013041,0.021579,0.021579,-8.427633e-08,0.023069,...,0.000067,0.000066,0.038076,0.000066,0.000067,0.000074,2.724538e-08,9.889398e-08,9.889398e-08,-0.000025


# Bulk Molecules

In [9]:
bulk_molecules = print_frames(table_to_frames( wildtype_simout / 'BulkMolecules'))

## atpRequested:

,Metabolism,RnaDegradation,TranscriptInitiation,TranscriptElongation,PolypeptideInitiation,PolypeptideElongation,ChromosomeReplication,ProteinDegradation,Complexation,ChromosomeFormation,Equilibrium,TfBinding,TwoComponentSystem
atpRequested,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,14048472,0,0,2106,0,0,0,0,0,0,112,0,731
2,14009828,0,0,2023,0,0,0,0,0,0,0,0,92
3,14002896,0,0,2030,0,0,0,0,0,0,0,0,0
4,13995878,0,0,2022,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
672,16515784,0,0,11013,0,0,0,0,0,0,0,0,0
673,16519588,0,0,11168,0,0,0,0,0,0,0,0,9
674,16523386,0,0,11261,0,0,0,0,0,0,0,0,10


## atpAllocatedFinal:

,Metabolism,RnaDegradation,TranscriptInitiation,TranscriptElongation,PolypeptideInitiation,PolypeptideElongation,ChromosomeReplication,ProteinDegradation,Complexation,ChromosomeFormation,Equilibrium,TfBinding,TwoComponentSystem
atpAllocatedFinal,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,7004183,0,0,0,0,0,0,0,0,0,0,0,731
2,7001448,0,0,0,0,0,0,0,0,0,0,0,0
3,6997938,0,0,0,0,0,0,0,0,0,0,0,1
4,6993763,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
672,8259794,0,0,0,0,0,0,0,0,0,0,0,0
673,8261684,0,0,0,0,0,0,0,0,0,0,0,9
674,8263585,0,0,0,0,0,0,0,0,0,0,0,0


## atpAllocatedInitial:

,Metabolism,RnaDegradation,TranscriptInitiation,TranscriptElongation,PolypeptideInitiation,PolypeptideElongation,ChromosomeReplication,ProteinDegradation,Complexation,ChromosomeFormation,Equilibrium,TfBinding,TwoComponentSystem
atpAllocatedInitial,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,7021287,0,0,2106,0,0,0,0,0,0,112,0,731
2,7002799,0,0,2023,0,0,0,0,0,0,0,0,92
3,6999418,0,0,2030,0,0,0,0,0,0,0,0,0
4,6995916,0,0,2022,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
672,8246879,0,0,11013,0,0,0,0,0,0,0,0,0
673,8248617,0,0,11168,0,0,0,0,0,0,0,0,9
674,8250422,0,0,11261,0,0,0,0,0,0,0,0,10


## counts:

,CPD-205[n],CPD-205[j],CPD-205[w],CPD-205[c],CPD-205[e],CPD-205[m],CPD-205[o],CPD-205[p],CPD-205[l],CPD-205[i],...,trpT-tRNA__alpha,tyrT-tRNA__alpha,tyrU-tRNA__alpha,tyrV-tRNA__alpha,valT-tRNA__alpha,valU-tRNA__alpha,valV-tRNA__alpha,valW-tRNA__alpha,valX-tRNA__alpha,valY-tRNA__alpha
counts,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
1,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
2,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
3,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
4,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
672,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
673,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
674,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1


In [8]:
unique_molecules = print_table( wildtype_simout / 'UniqueMolecules')

NameError: name 'print_table' is not defined